# Type별로 키워드 전환

In [1]:
import os, json

from tools import ONEDRIVE_PATH

ONEDRIVE_NEW = os.path.join(f'{ONEDRIVE_PATH}'.replace('selectstar', '데이터검수'), '재설계', '1. Q_type_label')
origin_type = json.load(open(os.path.join(ONEDRIVE_NEW, 'E1_금융일반_L.json'), 'r'))

In [2]:
# 분류 유형
# - a : 단일 개념에 대한 부정형
# - b : 단일 개념에 대한 긍정형
# - c : 박스형 / 조합형
# - d : 계산 및 수치 도출형
# - e : 용어 정의 및 지식 확인형
# - f : 빈칸 채우기 및 순서 나열형

types = {}
for q in origin_type:
    q_type = q.get('type', 'Unknown')
    if q_type in types:
        types[q_type].append(q)
    else:
        types[q_type] = [q]

# 결과 출력
print(f"총 문제 수: {len(origin_type)}")
print(f"\nType별 분류:")
for type_name, count in sorted(types.items()):
    print(f"  {type_name}: {len(count)}개")

총 문제 수: 250

Type별 분류:
  a: 107개
  b: 52개
  c: 28개
  d: 33개
  e: 22개
  f: 8개


In [ ]:
# 계산 및 수치 도출형 - 얼마인가? 계산하시오
# 용어 정의 및 지식 확인형 - 설명하시오
# 빈칸 채우기 및 순서 나열형 - 채워넣으시오, 나열하시오

In [3]:
with open('exam1_cal.json', 'w', encoding='utf-8') as f:
    json.dump(types['d'], f, ensure_ascii=False, indent=4)

In [4]:
from tools.transformed import *
from tqdm import tqdm

llm, onedrive_path, log_func = _init_common()
input_file = 'exam1_cal.json'
questions = _load_questions(input_file, log_func, '1단계')

1단계 입력 파일 읽기: exam1_cal.json
총 33개의 문제 처리 시작


In [6]:
for q in questions:
    print(q['question'])
    print("==============")

다음은 세 경기자(1, 2, 3)의 전략 선택에 따라 결정되는 보수구조이다. 개별 경기자가 선택할 수 있는 전략이 L과 H라고 할 때, 아래 전략형게임의 순수전략 내쉬균형은 몇 개인가? (단, 보수행렬의 괄호 안 첫 번째 숫자는 경기자 1의 보수, 두 번째 숫자는 경기자 2의 보수, 세 번째 숫자는 경기자 3의 보수를 나타낸다.)
\begin{tabular}{|c|c|c|c|}
\multicolumn{2}{c}{} & \multicolumn{2}{c}{{경기자 2}} \\
\cline{3-4}
\multicolumn{2}{c|}{} & \textit{L} & \textit{H} \\
\cline{2-4}
\multicolumn{1}{c|}{\multirow{2}{*}{{경기자 1}}} & \textit{L} & (2, 2, 2) & (2, 5, 2) \\
\cline{2-4}
\multicolumn{1}{c|}{} & \textit{H} & (5, 2, 2) & (5, 5, 2) \\
\cline{2-4}
\end{tabular}
\begin{tabular}{|c|c|c|c|}
\multicolumn{2}{c}{} & \multicolumn{2}{c}{{경기자 2}} \\
\cline{3-4}
\multicolumn{2}{c|}{} & \textit{L} & \textit{H} \\
\cline{2-4}
\multicolumn{1}{c|}{\multirow{2}{*}{{경기자 1}}} & \textit{L} & (2, 2, 5) & (2, 5, 5) \\
\cline{2-4}
\multicolumn{1}{c|}{} & \textit{H} & (5, 2, 5) & (0, 0, 3) \\
\cline{2-4}
\end{tabular}
대표적 소비자의 생애효용함수가 다음과 같다.
\[U(c_1,\; c_2)=\sqrt{c_1}+\frac{1}{1+r}\sqrt{c_2}\]
이 소비자는 1기에 근로소득 y를 얻는 반면, 2기에는 근로소득이 없다. 이 소비자가 1기에 

In [ ]:
# 서술형 문제로 변환
system_prompt = """당신은 25년 경력의 서술형 문제 전문가입니다. 아래 지시사항을 정확히 이해하고 수행하여 서술형 문제로 변환하시오.

지시사항:
1. 주어진 question에서 계산하려는 대상을 식별하라.
2. 계산하려는 대상을 바탕으로 계산 및 수치 도출형 문제로 변환하라.
3. 수식이나 표, 조건 등은 모두 유지하라. 
예시:
    - 수요의 가격탄력성에 관한 설명으로 옳지 않은 것은? (단, Q는 수량, P는 가격이다.) -> 다음 키워드를 활용하여 수요의 가격탄력성에 대해 서술하시오. (단, Q는 수량, P는 가격이다.)
    - 소비자 행동 이론 중 '고관여 제품'의 특징으로 옳지 않은 것은? -> 다음 키워드를 활용하여 소비자 행동 이론 중 '고관여 제품'의 특징에 대해 서술하시오.
    - 다음 중 금융투자회사의 투자설명서 이외에 추가로 핵심설명서 교부대상이 아닌 것은? -> 다음 키워드를 활용하여 금융투자회사의 투자설명서 이외에 추가로 핵심설명서 교부대상에 대해 서술하시오.
    - 다음 중 IS곡선상의 이동이 아닌 IS곡선 자체의 이동과 관련이 없는 것은? -> 다음 키워드를 활용하여 IS곡선상의 이동이 아닌 IS곡선 자체의 이동에 대해 서술하시오.

출력 형식:
- 서술형 문제: 다음 키워드를 활용하여 [주제]에 대해 서술하시오. (단서조항, 수식, 조건, 표 등 모두 유지)
"""
log_func("서술형 문제 변환 중...")
for q in tqdm(questions, desc="서술형 문제 변환"):
    _clean_question_data(q)
    
    user_prompt = f"""
    입력:
    - question: {q['question']}

    """
    response = llm.query_openrouter(system_prompt, user_prompt, model_name='google/gemini-2.5-flash')
    q['essay_question'] = response.replace('서술형 문제: ', '').replace("[", "").replace("]", "").replace("-", "").strip()